# **Azure AI Agents Service**

## Overview
Azure AI Agents Service enables you to create and deploy intelligent agents that understand natural language, perform actions, and help users accomplish tasks. These agents integrate with your data sources and services through custom functions, providing powerful conversational AI capabilities.

## Key Concepts

### 1. Agent Functions
Functions extend your agent's abilities by letting it:
- Search the web for information (demonstrated with Tavily search integration)
- Access external data sources (like weather information)
- Execute custom business logic
- Interact with your organization's systems and APIs

### 2. Agent Creation and Configuration
When creating your agent:
- Select an AI model (like GPT-4o)
- Define custom instructions to shape personality and behavior
- Register function tools that the agent can use
- Configure authentication and security settings

### 3. Threads and Messages
Conversations with agents happen within threads:
- Create a thread to establish a conversation context
- Add user messages to the thread
- Run the agent to process messages and generate responses

### 4. Tool Execution Flow
When an agent needs information:
1. The agent identifies when to use a tool
2. Your application receives the tool call request
3. Your code executes the appropriate function
4. You submit results back to the agent
5. The agent incorporates the results into its response

## Resources
* [Azure AI Agents documentation](https://learn.microsoft.com/azure/ai-services/azure-ai-agent)
* [AI Studio](https://ai.azure.com)

**Imports**

In [1]:
import os
import dotenv
dotenv.load_dotenv(".env")

True

**Create Client**

In [2]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

**Action Tools**

In [ ]:
import json
from typing import Any, Callable, Set, Dict, List, Optional
from tavily import TavilyClient

def tavily_web_search(search_query: str, max_results: int = 3) -> List[Dict[str, Any]]:
    """
    Performs a web search using Tavily and returns the results.

    :param search_query (str): The query string to search for.
    :param max_results (int): Maximum number of results to return. Defaults to 3.
    :return: List of search result dictionaries.
    :rtype: List[Dict[str, Any]]
    """
    try:
        tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
        response = tavily_client.search(search_query, max_results=max_results)

        if isinstance(response, dict):
            return response
        else:
            return json.loads(response)
    except Exception as e:
        return {"error": str(e), "results": []}
    
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json


# Statically defined user functions for fast reference
agent_functions: Set[Callable[..., Any]] = {
    tavily_web_search,
    fetch_weather
}


**Create Agent**

In [7]:
project_client.agents._config.endpoint

'https://swedencentral.api.azureml.ms'

In [4]:
from azure.ai.projects.models import FunctionTool

functions = FunctionTool(functions=agent_functions)

agent = project_client.agents.create_agent(
    name="my-action-tool-agent",
    model=os.getenv("chatModel"),
    instructions="You are a helpful assistant",
    tools=functions.definitions,
)

print(f"Created agent, ID: {agent.id}")

ResourceNotFoundError: () Service invocation failed!
Request: GET account-rp.vienna-swedencentral.svc/rp/workspaces/subscriptions/4ce7ad8d-95ed-4652-bd4a-5f2af19d29cb/resourceGroups/rg-ericsson-learning-path/providers/Microsoft.MachineLearningServices/workspaces/ericsson-learning-path-project/connections/aoaiericssonlearningpath
Status Code: 404 NotFound
Error Code: UserError/NotFoundError
Reason Phrase: Connection aoaiericssonlearningpath can't be found in this workspace
Response Body: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "Connection aoaiericssonlearningpath can't be found in this workspace",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null,
    "additionalInfo": null
  },
  "correlation": {
    "operation": "e116e1456ff23050f4b0a34cc5aa3a7

**Create Thread & Adding Message**

In [5]:
# user_message = "hi, what's the weather in New York?"
user_message = "Do a web search for latest news in AI"
# user_message = "What is the latest news in AI when it comes to Google?"

In [6]:
# with project_client:
thread = project_client.agents.create_thread()
agent = project_client.agents.get_agent(agent_id=agent.id)

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_c4CRoDogdtgUwzobHIS1bFNN


**Running Agent with Thread**

In [7]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput

run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        # When the status is requires_action, your code is responsible for calling the function tools.
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls: # If no tool calls are provided, cancel the run.
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    # If the output is not a string, convert it to JSON string
                    if not isinstance(output, str):
                        output = json.dumps(output)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )
    print(f"Current run status: {run.status}")
print(f"Run completed with status: {run.status}")

Created run, ID: run_3iBQKjFcTGDUT7XGJAbTODMT
Executing tool call: {'id': 'call_GwLsKcuEVik349c3Vpt7bP35', 'type': 'function', 'function': {'name': 'tavily_web_search', 'arguments': '{\n  "search_query": "latest news in AI",\n  "max_results": 3\n}'}}
Tool outputs: [{'tool_call_id': 'call_GwLsKcuEVik349c3Vpt7bP35', 'output': '{"query": "latest news in AI", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Artificial Intelligence News -- ScienceDaily", "url": "https://www.sciencedaily.com/news/computers_math/artificial_intelligence/", "content": "Mar. 18, 2025 \\u2014 Virginia Tech researchers say a true revolution in wireless technologies is only possible through endowing the system with the next generation of artificial intelligence (AI) that can think, imagine, and plan akin to humans. Coffee-Making Robot Breaks New Ground for AI Machines Mar. 18, 2025 \\u2014 An AI-powered robot that can prepare cups of coffee in a busy kitchen could usher in the next 

In [8]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)

for m in reversed(messages.data):
    if m.content[0].text.annotations:
        print(f"Message ({m.role}): {m.content[0].text.value}")
        for ref in m.content[0].text.annotations:
            print(f"Citations: {ref.url_citation.title} - {ref.url_citation.url}")
    else:
        print(f"Message ({m.role}): {m.content[0].text.value}")

Message (MessageRole.USER): Do a web search for latest news in AI
Message (MessageRole.AGENT): Here are the latest news articles in AI:

1. **[Artificial Intelligence News -- ScienceDaily](https://www.sciencedaily.com/news/computers_math/artificial_intelligence/)**:
   - Virginia Tech researchers say a true revolution in wireless technologies is only possible through endowing the system with the next generation of AI that can think, imagine, and plan akin to humans.
   - Coffee-Making Robot Breaks New Ground for AI Machines.
   - Researchers have enabled a man who is paralyzed to control a robotic arm through a device that relays signals from his brain to a computer.
   - Development of a new AI algorithm, called Torque Clustering, that significantly improves how AI systems independently learn and uncover patterns in data, without human intervention.

2. **[AI News & Artificial Intelligence | TechCrunch](https://techcrunch.com/category/artificial-intelligence/)**:
   - People are using

**Delete Agent & Thread**

In [9]:
# Delete the agent when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

# Delete Thread when done
project_client.agents.delete_thread(thread_id=thread.id)
print(f"thread: {thread.id} deleted")

Deleted agent
thread: thread_MbuJarQEWKvRHUCOQiGwo9PP deleted
